In [ ]:
# Starting Chrome Session
from selenium import webdriver
browser = webdriver.Chrome("path-to-chromedriver")
browser.get('https://drivers.uber.com/p3/fleet-manager/trips')

In [ ]:
# Placing Username
username = browser.find_element_by_id('useridInput')
username.send_keys('enter-your-email-here')     #Enter your username over here
Next = browser.find_element_by_class_name('push-small--right')
Next.click()

In [ ]:
# Placing Password
password = browser.find_element_by_id('password')
password.send_keys('enter-your-password-here')     #Enter your password over here
Next = browser.find_element_by_class_name('push-small--right')
Next.click()

In [ ]:
# Placing Verification Code (Only use this if uber asks for a verification code page on chrome)
verification = browser.find_element_by_id('verificationCode')
verification.send_keys('enter-your-verification-code-here')
Next = browser.find_element_by_class_name('push-small--right')
Next.click()

# Uber Fleet Manager Scrapper

In [ ]:
# Define all necessary arrays
timeseriesPD = []
farePD = []
durPD = []
disPD = []
statusPD = []
shopPD = []
droptimePD = []
droplocPD = []
orderedPD = []
shopPD = []
pickedPD = []
deliveredPD = []
droplocPD = []

In [ ]:
# Make necessary imports
from bs4 import BeautifulSoup
import time

# Detect the number of pages in the week
soup = BeautifulSoup(browser.page_source, 'html.parser')
num_pages = soup.find_all("nav", role="navigation")[0].get_text().split('Previous')[1].split('Next')[0][-1]
print("This week has a total of " + str(num_pages) + " pages.")
loop = int(num_pages) + 2

# Run the loop for reaching all of the pages

for x in range(2,loop):
    regixNP = "//nav[@role='navigation']/ul/li[" + str(x) + "]"
    browser.find_element_by_xpath(regixNP).click()
    time.sleep(4)
    
    new_soup = BeautifulSoup(browser.page_source, 'html.parser')
    rows = new_soup.find_all("tr", class_="cursor--pointer _style_2T0IvR")
    disp0 = "Page " + str(x-1) + " has " + str(len(rows)) + " rows."
    print(disp0)
    n=1

# Run the loop to open all the rows to open all the data
    for y in range(1,len(rows)*2,2):
        regixR = "//table[1]/tbody/tr[" + str(y) + "]/td/i"
        browser.find_element_by_xpath(regixR).click()
        disp1 = "For page " + str(x-1) + " processing row " + str(n) + "/" + str(len(rows))
        print(disp1)
        n+=1
        
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
    time.sleep(1)
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    print("Scrapping operation started.")
    
# Run a loop to scrap all data
    for z in range(0,len(rows)*2):
        
# Scrap all the data and write a condition for even and odd ranges
        if z%2 == 0:
            ss = soup.select("tbody[data-reactid*='366'] tr")[z].get_text()
            datetime = ss.split(' Requested: ')[1].split(' AESTDriver:\xa0')[0]
            driver = ss.split(' Requested: ')[1].split(' AESTDriver:\xa0')[1].split(' License: ')[0]
            car = ss.split(' Requested: ')[1].split(' AESTDriver:\xa0')[1].split(' License: ')[-1].split(' Fare: ')[0]
            fare = ss.split(' Requested: ')[1].split(' AESTDriver:\xa0')[1].split(' License: ')[-1].split(' Fare: ')[-1].split(' Time:   ')[0]
            dur = ss.split(' Requested: ')[1].split(' AESTDriver:\xa0')[1].split(' License: ')[-1].split(' Fare: ')[-1].split(' Time:   ')[-1].split(' Kilometers: ')[0]
            dis = ss.split(' Requested: ')[1].split(' AESTDriver:\xa0')[1].split(' License: ')[-1].split(' Fare: ')[-1].split(' Time:   ')[-1].split(' Kilometers: ')[-1].split(' Status: ')[0]
            status = ss.split(' Requested: ')[1].split(' AESTDriver:\xa0')[1].split(' License: ')[-1].split(' Fare: ')[-1].split(' Time:   ')[-1].split(' Kilometers: ')[-1].split(' Status: ')[-1]
            
            timeseriesPD.append(datetime)
            farePD.append(fare)
            durPD.append(dur)
            disPD.append(dis)
            statusPD.append(status)
            
        elif z%2 == 1:
            tt = soup.select("tbody[data-reactid*='366'] tr")[z].get_text()
            timeordered = tt.split('\n\n\n\n\n\n            DELIVERY\n          \n\n\n\n\n\n\n\n          ')[-1].split(' AEST\n        \n\n          ')[0]
            shop = tt.split('\n\n\n\n\n\n            DELIVERY\n          \n\n\n\n\n\n\n\n          ')[-1].split(' AEST\n        \n\n          ')[1].split('\n        \n\n\n\n\n\n\n\n\n\n          ')[0]
            timepicked = tt.split('\n\n\n\n\n\n            DELIVERY\n          \n\n\n\n\n\n\n\n          ')[-1].split(' AEST\n        \n\n          ')[1].split('\n        \n\n\n\n\n\n\n\n\n\n          ')[-1]
            timedelivered = tt.split('\n\n\n\n\n\n            DELIVERY\n          \n\n\n\n\n\n\n\n          ')[-1].split(' AEST\n        \n\n          ')[2].split('\n        \n\n\n\n\n\n\n\n\n\n          ')[-1]
            droploc = tt.split('\n\n\n\n\n\n            DELIVERY\n          \n\n\n\n\n\n\n\n          ')[-1].split(' AEST\n        \n\n          ')[-1].split('\n        \n\n\n\n')[0]
            
            orderedPD.append(timeordered)
            shopPD.append(shop)
            pickedPD.append(timepicked)
            deliveredPD.append(timedelivered)
            droplocPD.append(droploc)
        
        else:
            print("Error.")
            
    print("Page " + str(x-1) + " scrapping Complete.")


In [ ]:
import pandas as pd
scrapped = { "Date and Time" : timeseriesPD,
             "Fare" : farePD,
             "Duration" : durPD,
             "Distance" : disPD,
             "Status" : statusPD,
             "Resturant" : shopPD,
             "Drop Location" : droplocPD,
             "Time Ordered" : orderedPD,
             "Time Picked" : pickedPD,
             "Time Delivered" : deliveredPD
           }

uber_trip = pd.DataFrame(scrapped)
uber_trip.to_csv('path-to-output-file')
uber_trip

In [ ]:
uber_trip